In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import mchmm as mc
# https://github.com/maximtrp/mchmm

In [2]:
df = pd.read_csv('szczecin_sep_2009.txt', index_col=False, header=None)
selected_cols = df[[1, 3, 4, 7, 11]]
df = selected_cols.copy(deep=True)
df.columns = ['temp', 'wilg', 'cisn', 'wiatr','stan']

#temperatura (kolumna 2), wilgotność (kolumna 4), ciśnienie (kolumna 5),
#prędkość wiatru (po dyskretyzacji) (kolumna 8)

Zredukowanie liczby stanów pogodowych.

In [3]:
df['wiatr'].replace({'Calm':1.0},  inplace=True)
df = df[pd.to_numeric(df['wiatr'], errors='coerce').notnull()]

def to_float_(x):
    return float(x)

df["wiatr"] = df["wiatr"].apply(to_float_)

In [4]:
df['temp'] = pd.cut(df['temp'], bins=4, labels=[1, 2, 3, 4])
df['wilg'] = pd.cut(df['wilg'], bins=4, labels=[1, 2, 3, 4])
df['cisn'] = pd.cut(df['cisn'], bins=4, labels=[1, 2, 3, 4])
df['wiatr'] = pd.cut(df['wiatr'], bins=4, labels=[1, 2, 3, 4])

df.head(3)

,temp,wilg,cisn,wiatr,stan
0,2,3,3,4,Clear
1,2,3,3,4,Clear
2,2,3,3,4,Clear


Podział na dane uczące, testujące

In [5]:
obserwacje = []
for i in range(df.shape[0]):
    xx = df.iloc[i][['temp','wilg','cisn' ,'wiatr']]
    a = ''
    for j in range(xx.shape[0]):        
        a = a + (str(xx[j]))
    obserwacje.append(a)

T_ = [12, 24, 48]
T = T_[0]

Y_ = df['stan'].to_numpy()

X = []
Y = []

i = T
while i < len(obserwacje):
    X.append(obserwacje[i-T:i])
    Y.append(Y_[i])
    i += T

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    obserwacje,Y_, test_size=0.3, random_state=42)

## Viterbi

Stworzenie modelu

In [7]:
obs_seq, sts_seq = X_train, y_train

a = mc.HiddenMarkovModel().from_seq(obs_seq, sts_seq)

In [8]:
#pd.DataFrame(a.ep, index=a.states, columns=a.observations)

6. Przy użyciu algorytmu Viterbiego należy odkryć ukrytą ścieżkę stanów
pogodowych (”Condition”) mając daną sekwencję obserwacji.

In [9]:
liczba_iteracji = 0
poprawne = 0

i = T
while i < len(X_test):
    sek = X_test[i-T:i]
    res = a.viterbi(sek)
    popr = y_test[i-T:i]
    for j in range(len(popr)):
        if popr[j] == res[0][j]:
            poprawne += 1
    i += T
    liczba_iteracji += 1

print('Poprawne: ',poprawne, ', łącznie: ', liczba_iteracji*T)
print('Poprawność: ', (poprawne * 100) / (liczba_iteracji * T), '%')

Poprawne:  166 , łącznie:  420
Poprawność:  39.523809523809526 %


## Baum Welch

In [10]:
a = mc.HiddenMarkovModel().from_baum_welch(X_train, df['stan'].unique() )

In [11]:
liczba_iteracji = 0
poprawne = 0

i = T
while i < len(X_test):
    sek = X_test[i-T:i]
    res = a.viterbi(sek)
    popr = y_test[i-T:i]
    for j in range(len(popr)):
        if popr[j] == res[0][j]:
            poprawne += 1
    i += T
    liczba_iteracji += 1

print('Poprawne: ',poprawne, ', łącznie: ', liczba_iteracji*T)
print('Poprawność: ', (poprawne * 100) / (liczba_iteracji * T), '%')

Poprawne:  74 , łącznie:  420
Poprawność:  17.61904761904762 %


# Wnioski

Algorytm Bauma-Welscha jest mniej skuteczny niż Viterbi.